In [1]:
import os
import kagglehub
import pandas as pd

import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import time
import IPython.display as display
import tempfile
from sklearn.ensemble import AdaBoostClassifier
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit

## Execise 1

In this exercise, do the following:
1. Load the dataset used in the time series example - Energy consumption data. You can find it in the notebook "TSA_Example" in Time Series folder in Moodle.
2. Setup a nested MLFlow loop where different modelling experiments can be tracked and the use the dataset in point 1 to experiment and track models. You should do following combinations:
    1. At least 3 model types
    2. At least 3 different feature combinations
    3. At least 3 different options for 3 different hyperparameters
    4. At least 3 different time splits for train test
3. For each option in the combination, you should calculate & log the following in MLFlow:
    1. RMSE
    2. MAE
    3. Plot of actual vs predicted for 1 month data
    4. Plot of actual vs predicted for 1 week of data
    5. All of the combination info in point 2, such as which model, what feature combindation, what hyperparameter, what train test split has been used
4. Turn on MLFlow UI and track your experiments

1. Load the dataset used in the time series example - Energy consumption data. You can find it in the notebook "TSA_Example" in Time Series folder in Moodle.

In [2]:
# Download latest version
path = kagglehub.dataset_download("robikscube/hourly-energy-consumption")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Jacob\.cache\kagglehub\datasets\robikscube\hourly-energy-consumption\versions\3


In [3]:
dataset_path = "C:/Users/Jacob/.cache/kagglehub/datasets/robikscube/hourly-energy-consumption/versions/3"

files = os.listdir(dataset_path)
print(files)


['AEP_hourly.csv', 'COMED_hourly.csv', 'DAYTON_hourly.csv', 'DEOK_hourly.csv', 'DOM_hourly.csv', 'DUQ_hourly.csv', 'EKPC_hourly.csv', 'est_hourly.paruqet', 'FE_hourly.csv', 'NI_hourly.csv', 'PJME_hourly.csv', 'PJMW_hourly.csv', 'pjm_hourly_est.csv', 'PJM_Load_hourly.csv']


In [4]:
file_path = "C:/Users/Jacob/.cache/kagglehub/datasets/robikscube/hourly-energy-consumption/versions/3/PJME_hourly.csv"

In [5]:
df = pd.read_csv(file_path)

In [6]:
df

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0
...,...,...
145361,2018-01-01 20:00:00,44284.0
145362,2018-01-01 21:00:00,43751.0
145363,2018-01-01 22:00:00,42402.0
145364,2018-01-01 23:00:00,40164.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145366 entries, 0 to 145365
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Datetime  145366 non-null  object 
 1   PJME_MW   145366 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.2+ MB


In [8]:
duplicate_values = df[df["Datetime"].duplicated()]
print(f"We got this number of duplicates in Datetime col: {len(duplicate_values)}")

print(duplicate_values)


We got this number of duplicates in Datetime col: 4
                   Datetime  PJME_MW
106585  2014-11-02 02:00:00  23755.0
115369  2015-11-01 02:00:00  21171.0
124009  2016-11-06 02:00:00  21692.0
132817  2017-11-05 02:00:00  20666.0


In [9]:
df = df.drop_duplicates(subset="Datetime")


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145362 entries, 0 to 145365
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Datetime  145362 non-null  object 
 1   PJME_MW   145362 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.3+ MB


In [11]:
df = df.set_index("Datetime")
df.index = pd.to_datetime(df.index)
df = df.sort_index()

In [12]:
df.head()

,PJME_MW
Datetime,
2002-01-01 01:00:00,30393.0
2002-01-01 02:00:00,29265.0
2002-01-01 03:00:00,28357.0
2002-01-01 04:00:00,27899.0
2002-01-01 05:00:00,28057.0


Checking for missing timestamps.

In [13]:
range_of_dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq="h")
missing_timestamps = range_of_dates.difference(df.index)
missing_timestamps


DatetimeIndex(['2002-04-07 03:00:00', '2002-10-27 02:00:00',
               '2003-04-06 03:00:00', '2003-10-26 02:00:00',
               '2004-04-04 03:00:00', '2004-10-31 02:00:00',
               '2005-04-03 03:00:00', '2005-10-30 02:00:00',
               '2006-04-02 03:00:00', '2006-10-29 02:00:00',
               '2007-03-11 03:00:00', '2007-11-04 02:00:00',
               '2008-03-09 03:00:00', '2008-11-02 02:00:00',
               '2009-03-08 03:00:00', '2009-11-01 02:00:00',
               '2010-03-14 03:00:00', '2010-11-07 02:00:00',
               '2010-12-10 00:00:00', '2011-03-13 03:00:00',
               '2011-11-06 02:00:00', '2012-03-11 03:00:00',
               '2012-11-04 02:00:00', '2013-03-10 03:00:00',
               '2013-11-03 02:00:00', '2014-03-09 03:00:00',
               '2015-03-08 03:00:00', '2016-03-13 03:00:00',
               '2017-03-12 03:00:00', '2018-03-11 03:00:00'],
              dtype='datetime64[ns]', freq=None)

In [14]:
df.loc["2002-04-07 02:00:00":"2002-04-07 05:00:00"]

,PJME_MW
Datetime,
2002-04-07 02:00:00,24738.0
2002-04-07 04:00:00,24487.0
2002-04-07 05:00:00,24617.0


Adding missing dates:

In [15]:
df=df.reindex(range_of_dates)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145362 non-null  float64
dtypes: float64(1)
memory usage: 6.2 MB


In [17]:
df.loc["2002-04-07 02:00:00":"2002-04-07 05:00:00"]

,PJME_MW
2002-04-07 02:00:00,24738.0
2002-04-07 03:00:00,NaN
2002-04-07 04:00:00,24487.0
2002-04-07 05:00:00,24617.0


Filling missing values that came due to the above

In [18]:
df = df.fillna(method="ffill")

C:\Users\Jacob\AppData\Local\Temp\ipykernel_1216\567689999.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")


In [19]:
df.loc["2002-04-07 02:00:00":"2002-04-07 05:00:00"]

,PJME_MW
2002-04-07 02:00:00,24738.0
2002-04-07 03:00:00,24738.0
2002-04-07 04:00:00,24487.0
2002-04-07 05:00:00,24617.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145392 non-null  float64
dtypes: float64(1)
memory usage: 6.2 MB


In [21]:
df["hour"] = df.index.hour
df["day"] = df.index.day
df["month"] = df.index.month
df["weekday"] = df.index.weekday

In [22]:
df.head()

,PJME_MW,hour,day,month,weekday
2002-01-01 01:00:00,30393.0,1,1,1,1
2002-01-01 02:00:00,29265.0,2,1,1,1
2002-01-01 03:00:00,28357.0,3,1,1,1
2002-01-01 04:00:00,27899.0,4,1,1,1
2002-01-01 05:00:00,28057.0,5,1,1,1


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145392 non-null  float64
 1   hour     145392 non-null  int32  
 2   day      145392 non-null  int32  
 3   month    145392 non-null  int32  
 4   weekday  145392 non-null  int32  
dtypes: float64(1), int32(4)
memory usage: 8.5 MB


In [24]:
# def create_features(df):

#     df = df.copy()
#     # Lag features
#     df['lag_1d'] = df['PJME_MW'].shift(1*24)   # 1 day lag
#     df['lag_1w'] = df['PJME_MW'].shift(7*24)   # 1 week lag
#     df['lag_1m'] = df['PJME_MW'].shift(30*24)  # 1 month lag

#     # Rolling statistics features
#     df['rolling_mean_1d'] = df['PJME_MW'].rolling(window=1*24).mean()  # Last 1 day rolling mean
#     df['rolling_mean_7d'] = df['PJME_MW'].rolling(window=7*24).mean()  # Last 7 days rolling mean
#     df['rolling_mean_30d'] = df['PJME_MW'].rolling(window=30*24).mean()  # Last 30 days rolling mean
#     df['rolling_mean_90d'] = df['PJME_MW'].rolling(window=90*24).mean()  # Last 90 days rolling mean
#     df['rolling_mean_same_month_last_year'] = df['PJME_MW'].shift(365*24).rolling(window=30*24).mean()  # Same month previous year rolling mean
#     df['rolling_mean_same_week_last_year'] = df['PJME_MW'].shift(365*24).rolling(window=7*24).mean()  # Same week previous year rolling mean
#     df['rolling_mean_same_day_last_year'] = df['PJME_MW'].shift(365*24).rolling(window=1*24).mean()  # Same day previous year rolling mean

#     return df

# df = create_features(df)

In [25]:
# # Select the first combination
# X = df.drop(columns=["PJME_MW"])
# y = df['PJME_MW']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Define different feature combinations
# # Define 3 feature combinations based on your dataset
# feature_combinations = [
#     ["hour", "day", "month"],  # Combination 1: Time-based features (hour, day, month)
#     ["lag_1d", "lag_1w", "lag_1m"],  # Combination 2: Lag features (1-day, 1-week, 1-month lags)
#     ["rolling_mean_1d", "rolling_mean_7d", "rolling_mean_30d"],  # Combination 3: Rolling averages (short-term and long-term)
# ]

2. Setup a nested MLFlow loop where different modelling experiments can be tracked and the use the dataset in point 1 to experiment and track models. You should do following combinations:
    1. At least 3 model types
    2. At least 3 different feature combinations
    3. At least 3 different options for 3 different hyperparameters
    4. At least 3 different time splits for train test

In [26]:
feature_combinations = [
    ["hour", "day", "month", "weekday"],
    ["hour", "day", "month"],
    ["hour", "day", "weekday"],
]

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145392 non-null  float64
 1   hour     145392 non-null  int32  
 2   day      145392 non-null  int32  
 3   month    145392 non-null  int32  
 4   weekday  145392 non-null  int32  
dtypes: float64(1), int32(4)
memory usage: 8.5 MB


In [28]:
models = {
    "LinearRegression": LinearRegression,
    "RandomForest": RandomForestRegressor,
    "XGBoost": XGBRegressor
}

In [29]:
# hyperparameter_options = {
#     "n_estimators": [50, 100, 150],
#     "max_depth": [5, 10, 15],
#     "learning_rate": [0.01, 0.1, 0.2]
# }

In [30]:
hyperparameter_options = {
    "n_estimators": [50],
    "max_depth": [5],
    "learning_rate": [0.01]
}

# To do, date +1 hour off!!

In [ ]:
# Training splits for our data, 60, 45 and 30 days
split_points = [df.index[-(60*24)], df.index[-(45*24)], df.index[-(30*24)]] 

In [ ]:
# Start MLflow experiment
mlflow.set_experiment("tester2")

for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name): 
        for features in feature_combinations:
            feature_name = "-".join(features)
            for n_estimators in hyperparameter_options["n_estimators"]:
                for max_depth in hyperparameter_options["max_depth"]:
                    for learning_rate in hyperparameter_options["learning_rate"]:
                        for split_point in split_points:
                            child_run_name = f"{feature_name}| n_est={n_estimators}| depth={max_depth}| lr={learning_rate}| time_split_point={split_point}"
                            with mlflow.start_run(run_name=child_run_name, nested=True):

                                X = df[features]
                                y = df["PJME_MW"]

                                # For X_train, slice until (excluding split_point)
                                X_train, X_test = X[:split_point], X[split_point:]
                                y_train, y_test = y[:split_point], y[split_point:]

                                # Model training and hyperparameter tuning using validation set
                                if model_name == "LinearRegression":
                                    model = LinearRegression(n_jobs=-1)
                                elif model_name == "RandomForest":
                                    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1)
                                elif model_name == "XGBoost":
                                    model = XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, n_jobs=-1)

                                model.fit(X_train, y_train)

                                y_test_pred = model.predict(X_test)

                                # Metrics on the test set
                                rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
                                mae_test = mean_absolute_error(y_test, y_test_pred)

                                # Log test metrics
                                mlflow.log_metric('RMSE_test', rmse_test)
                                mlflow.log_metric('MAE_test', mae_test)

                                # Log params
                                mlflow.log_param('model', model_name)
                                mlflow.log_param('features', feature_name)
                                mlflow.log_param('n_estimators', n_estimators)
                                mlflow.log_param('max_depth', max_depth)
                                mlflow.log_param('learning_rate', learning_rate)
                                mlflow.log_param('split_point', split_point)

                                # Plots for 1 week and 1 month on the test set
                                plt.figure(figsize=(8, 6))
                                plt.plot(y_test.index[:24*7], y_test[:24*7], label='Actual')
                                plt.plot(y_test.index[:24*7], y_test_pred[:24*7], label='Predicted')
                                plt.xlabel('Time')
                                plt.ylabel('Energy Consumption')
                                plt.title('1 Week - Actual vs Predicted')
                                plt.xticks(rotation=25, ha='right')
                                plt.legend()
                                plt.savefig('1_week_plot.png')
                                mlflow.log_artifact('1_week_plot.png')
                                plt.close()

                                plt.figure(figsize=(8, 6))
                                plt.plot(y_test.index[:24*30], y_test[:24*30], label='Actual')
                                plt.plot(y_test.index[:24*30], y_test_pred[:24*30], label='Predicted')
                                plt.xlabel('Time')
                                plt.ylabel('Energy Consumption')
                                plt.title('1 Month - Actual vs Predicted')
                                plt.xticks(rotation=25, ha='right')
                                plt.legend()
                                plt.savefig('1_month_plot.png')
                                mlflow.log_artifact('1_month_plot.png')
                                plt.close()

print('Experiment completed. Check MLflow UI for details.')

# To start MLflow UI, run this in terminal:
# mlflow ui


Experiment completed. Check MLflow UI for details.


In [33]:
# Start MLFlow UI from the notebook
print("\nStarting MLFlow UI...")
process = subprocess.Popen(["mlflow", "ui", "--port", "5000"])  # Starts MLFlow UI on port 5000

# Wait a moment to ensure the server starts
time.sleep(3)

# Display a link to the MLFlow UI
display.display(display.Markdown("[Open MLFlow UI](http://localhost:5000)"))

print("MLFlow UI is running at http://localhost:5000. Press Ctrl+C in the terminal to stop it.")


Starting MLFlow UI...


[Open MLFlow UI](http://localhost:5000)

MLFlow UI is running at http://localhost:5000. Press Ctrl+C in the terminal to stop it.
